# Tunability for XGBoostClassifier

# Intro

In [1]:
import pandas as pd

random_state = pd.read_csv("../../data/vars.csv")['random_state'].iloc[0]
n_iter = pd.read_csv("../../data/vars.csv")['n_iter'].iloc[0]
n_iter_BS = pd.read_csv("../../data/vars.csv")['n_iter_BS'].iloc[0]

In [2]:
df_college = pd.read_csv("../../data/processed/college.csv")
df_credit = pd.read_csv("../../data/processed/credit.csv")
df_diabetes = pd.read_csv("../../data/processed/diabetes.csv")
df_penguins = pd.read_csv("../../data/processed/penguins.csv")

In [3]:
print(df_college.info())
print()
print(df_credit.info())
print()
print(df_diabetes.info())
print()
print(df_penguins.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   type_school            1000 non-null   object 
 1   school_accreditation   1000 non-null   object 
 2   gender                 1000 non-null   object 
 3   interest               1000 non-null   object 
 4   residence              1000 non-null   object 
 5   parent_age             1000 non-null   int64  
 6   parent_salary          1000 non-null   int64  
 7   house_area             1000 non-null   float64
 8   average_grades         1000 non-null   float64
 9   parent_was_in_college  1000 non-null   bool   
 10  will_go_to_college     1000 non-null   int64  
dtypes: bool(1), float64(2), int64(3), object(5)
memory usage: 79.2+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 

In [4]:

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.impute import SimpleImputer
import xgboost as xgb
import numpy as np


num_pipeline = Pipeline(steps=[
    ('num_impute', SimpleImputer(strategy='mean')),
    ('scale', MinMaxScaler())
])

cat_pipeline = Pipeline(steps=[
    ('cat_impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot', OneHotEncoder(handle_unknown='ignore'))
])

col_trans = ColumnTransformer([
    ('num_pipeline', num_pipeline, make_column_selector(dtype_include = np.number)),
    ('cat_pipeline', cat_pipeline, make_column_selector(dtype_include = np.object_))
])

model_pipeline = Pipeline(steps=[
    ('preprocessing', col_trans),
    ('model', xgb.XGBClassifier())
])

# RandomizedSearchCV

In [5]:
from scipy.stats import uniform

xgboost_params = {
    'model__booster': ['gbtree', 'gblinear', 'dart'],  
    'model__learning_rate': uniform(0.0001, 0.3), # internet
    'model__max_depth': range(3, 11),  
    # 'min_child_weight': uniform(1, 10),  
    # 'subsample': uniform(0.6, 0.4),  
    # 'colsample_bytree': uniform(0.6, 0.4),  
    # 'gamma': uniform(0, 0.5),  
    'model__lambda': uniform(2**(-10), 2**10) # paper
    # 'alpha': uniform(0, 3), 
    # 'n_estimators': range(50, 301),  
}

## Credit dataset

In [6]:
from sklearn.model_selection import RandomizedSearchCV

X = df_credit.iloc[:, 0:-1]
y = df_credit.iloc[:, -1]

random_search_model = RandomizedSearchCV(estimator = model_pipeline,
                                   param_distributions = xgboost_params,
                                   n_iter = n_iter,
                                   cv = 5,
                                   scoring = 'f1',
                                   random_state = random_state)

random_search_model.fit(X, y)

best_f1_credit = random_search_model.best_score_

print(f"Best f1 score: {best_f1_credit}")
print(f"Best params: {random_search_model.best_params_}")

random_search_model.best_estimator_

c:\Users\natal\.conda\envs\tunability\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:44:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\natal\.conda\envs\tunability\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:44:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\natal\.conda\envs\tunability\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:44:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\natal\.conda\envs\t

Best f1 score: 0.8604659709273008
Best params: {'model__booster': 'dart', 'model__lambda': 871.3168355357284, 'model__learning_rate': 0.27222116369985394, 'model__max_depth': 3}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000017F2C35C290>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncod...
                               importance_type=None,
                               interaction_constraints=None,
                               lambda=871.3168355357284,
                               learning_rate=0.27222116369985394, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=3,
                               max_leaves=None, min_child_weight=None,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=None,
                               n_jobs=None, num_parallel_tree=None, ...))])

## Data frame with params, results and mean result

In [7]:
tested_params = random_search_model.cv_results_['params']

In [8]:
df_xgboost = pd.DataFrame(tested_params)
df_xgboost['credit_f1'] = random_search_model.cv_results_['mean_test_score']
df_xgboost

,model__booster,model__lambda,model__learning_rate,model__max_depth,credit_f1
0,dart,970.216479,0.062922,8,0.834125
1,dart,110.241417,0.178616,6,0.849342
2,gblinear,428.859783,0.100722,7,0.000000
3,dart,448.657797,0.220865,7,0.845668
4,gbtree,592.752183,0.193707,8,0.853093
...,...,...,...,...,...
195,gbtree,210.685896,0.174140,4,0.836696
196,gbtree,620.712871,0.175686,9,0.853098
197,dart,932.548532,0.034280,4,0.843928
198,gbtree,54.922199,0.050953,3,0.841125


## College dataset

In [9]:
from sklearn.model_selection import GridSearchCV

X = df_college.iloc[:, 0:-1]
y = df_college.iloc[:, -1]

formatted_params = []
for params in tested_params:
    formatted_param = {key: [value] for key, value in params.items()}
    formatted_params.append(formatted_param)

grid_search_model = GridSearchCV(model_pipeline,
                                 param_grid=formatted_params,
                                 scoring='f1',
                                 cv = 5)

grid_search_model.fit(X, y)

df_xgboost['college_f1'] = grid_search_model.cv_results_['mean_test_score']

best_f1_college = grid_search_model.best_score_

print(f"Best f1 score: {best_f1_college}")
print(f"Best params: {grid_search_model.best_params_}")
grid_search_model.best_estimator_

c:\Users\natal\.conda\envs\tunability\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:50:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\natal\.conda\envs\tunability\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:50:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\natal\.conda\envs\tunability\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:50:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\natal\.conda\envs\t

Best f1 score: 0.8959594606051887
Best params: {'model__booster': 'dart', 'model__lambda': 60.761012605254564, 'model__learning_rate': 0.25943094547969325, 'model__max_depth': 6}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000017F2D80E510>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncod...
                               importance_type=None,
                               interaction_constraints=None,
                               lambda=60.761012605254564,
                               learning_rate=0.25943094547969325, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=6,
                               max_leaves=None, min_child_weight=None,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=None,
                               n_jobs=None, num_parallel_tree=None, ...))])

## Diabetes dataset

In [10]:
from sklearn.model_selection import GridSearchCV

X = df_diabetes.iloc[:, 0:-1]
y = df_diabetes.iloc[:, -1]

formatted_params = []
for params in tested_params:
    formatted_param = {key: [value] for key, value in params.items()}
    formatted_params.append(formatted_param)

grid_search_model = GridSearchCV(model_pipeline,
                                 param_grid=formatted_params,
                                 scoring='f1',
                                 cv = 5)

grid_search_model.fit(X, y)

df_xgboost['diabetes_f1'] = grid_search_model.cv_results_['mean_test_score']

best_f1_diabetes = grid_search_model.best_score_

print(f"Best f1 score: {best_f1_diabetes}")
print(f"Best params: {grid_search_model.best_params_}")
grid_search_model.best_estimator_

c:\Users\natal\.conda\envs\tunability\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:55:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\natal\.conda\envs\tunability\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:55:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\natal\.conda\envs\tunability\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:55:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\natal\.conda\envs\t

Best f1 score: 0.6513630229419702
Best params: {'model__booster': 'dart', 'model__lambda': 60.761012605254564, 'model__learning_rate': 0.25943094547969325, 'model__max_depth': 6}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000017F2D85F710>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncod...
                               importance_type=None,
                               interaction_constraints=None,
                               lambda=60.761012605254564,
                               learning_rate=0.25943094547969325, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=6,
                               max_leaves=None, min_child_weight=None,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=None,
                               n_jobs=None, num_parallel_tree=None, ...))])

## Penguins dataset

In [11]:
from sklearn.model_selection import GridSearchCV

X = df_penguins.iloc[:, 0:-1]
y = df_penguins.iloc[:, -1]

formatted_params = []
for params in tested_params:
    formatted_param = {key: [value] for key, value in params.items()}
    formatted_params.append(formatted_param)

grid_search_model = GridSearchCV(model_pipeline,
                                 param_grid=formatted_params,
                                 scoring='f1',
                                 cv = 5)

grid_search_model.fit(X, y)

df_xgboost['penguins_f1'] = grid_search_model.cv_results_['mean_test_score']

best_f1_penguins = grid_search_model.best_score_


print(f"Best f1 score: {best_f1_penguins}")
print(f"Best params: {grid_search_model.best_params_}")
grid_search_model.best_estimator_

c:\Users\natal\.conda\envs\tunability\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:59:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\natal\.conda\envs\tunability\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:59:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\natal\.conda\envs\tunability\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:59:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\natal\.conda\envs\t

Best f1 score: 0.9899435028248588
Best params: {'model__booster': 'dart', 'model__lambda': 110.24141681693118, 'model__learning_rate': 0.17861561926187206, 'model__max_depth': 6}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000017F2BE1E850>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncod...
                               importance_type=None,
                               interaction_constraints=None,
                               lambda=110.24141681693118,
                               learning_rate=0.17861561926187206, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=6,
                               max_leaves=None, min_child_weight=None,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=None,
                               n_jobs=None, num_parallel_tree=None, ...))])

## Tunability 

In [12]:
df_xgboost['mean_f1'] = df_xgboost[['credit_f1', 'college_f1', 'diabetes_f1', 'penguins_f1']].mean(axis=1)
df_xgboost

,model__booster,model__lambda,model__learning_rate,model__max_depth,credit_f1,college_f1,diabetes_f1,penguins_f1,mean_f1
0,dart,970.216479,0.062922,8,0.834125,0.798016,0.000000,0.986665,0.654702
1,dart,110.241417,0.178616,6,0.849342,0.880575,0.641419,0.989944,0.840320
2,gblinear,428.859783,0.100722,7,0.000000,0.681882,0.000000,0.710573,0.348114
3,dart,448.657797,0.220865,7,0.845668,0.847328,0.600259,0.986665,0.819980
4,gbtree,592.752183,0.193707,8,0.853093,0.831755,0.566137,0.986665,0.809413
...,...,...,...,...,...,...,...,...,...
195,gbtree,210.685896,0.174140,4,0.836696,0.856815,0.623829,0.989944,0.826821
196,gbtree,620.712871,0.175686,9,0.853098,0.827381,0.561263,0.986665,0.807102
197,dart,932.548532,0.034280,4,0.843928,0.784434,0.000000,0.710573,0.584734
198,gbtree,54.922199,0.050953,3,0.841125,0.845801,0.620478,0.989944,0.824337


In [13]:
# sigma star
sigma_star_row = df_xgboost[df_xgboost['mean_f1'] == max(df_xgboost['mean_f1'])]
sigma_star_row

,model__booster,model__lambda,model__learning_rate,model__max_depth,credit_f1,college_f1,diabetes_f1,penguins_f1,mean_f1
55,dart,60.761013,0.259431,6,0.847078,0.895959,0.651363,0.989944,0.846086


### (3.3) Measuring Overall Tunability of a ML Algorithm

In [14]:
d_credit = -sigma_star_row['credit_f1'].iloc[0] + best_f1_credit
d_college = -sigma_star_row['college_f1'].iloc[0] + best_f1_college
d_diabetes = -sigma_star_row['diabetes_f1'].iloc[0] + best_f1_diabetes
d_penguins = -sigma_star_row['penguins_f1'].iloc[0] + best_f1_penguins

d = pd.Series([d_credit, d_college, d_diabetes, d_penguins])
print(d)
print(f"Mean: {d.mean()}")


0    0.013388
1    0.000000
2    0.000000
3    0.000000
dtype: float64
Mean: 0.0033469929181471902


### Saving results

In [15]:
df_xgboost.to_csv("../../data/RSmodels/xgb_results.csv", index=False)

# BayesSearchCV

In [32]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

n_iter_BS = pd.read_csv('../../data/vars.csv')['n_iter_BS'].iloc[0]
BS_iterations = pd.read_csv('../../data/vars.csv')['BS_iterations'].iloc[0]

xgboost_params = {
    'model__booster': Categorical(['gbtree', 'gblinear', 'dart']),  
    'model__learning_rate': Real(0.0001, 0.3),  
    'model__max_depth': Integer(3, 10),  
    'model__lambda': Real(2**(-10), 2**10),  
}

## Credit dataset

In [33]:
X = df_credit.iloc[:, 0:-1]
y = df_credit.iloc[:, -1]


bayes_search_credit_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=xgboost_params,
                                n_iter=int(n_iter_BS),
                                #random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_credit_model.fit(X, y)

best_BS_f1_credit = bayes_search_credit_model.best_score_
credit_best_estimator = bayes_search_credit_model.best_estimator_

print(f"Best f1 score: {best_BS_f1_credit}")

credit_best_estimator

Best f1 score: 0.8604659709273008


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000282CD97B4D0>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncod...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, lambda=1024.0,
                               learning_rate=0.3, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=3,
                               max_leaves=None, min_child_weight=None,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=None,
                               n_jobs=None, num_parallel_tree=None, ...))])

## College dataset

In [34]:
X = df_college.iloc[:, 0:-1]
y = df_college.iloc[:, -1]


bayes_search_college_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=xgboost_params,
                                n_iter=int(n_iter_BS),
                                #random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_college_model.fit(X, y)

best_BS_f1_college = bayes_search_college_model.best_score_
college_best_estimator = bayes_search_college_model.best_estimator_

print(f"Best f1 score: {best_BS_f1_college}")

college_best_estimator

Best f1 score: 0.9033880244415696


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000282CB148590>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncod...
                               importance_type=None,
                               interaction_constraints=None,
                               lambda=3.5715800230084014,
                               learning_rate=0.29920051667283853, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=4,
                               max_leaves=None, min_child_weight=None,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=None,
                               n_jobs=None, num_parallel_tree=None, ...))])

## Diabetes dataset

In [35]:
X = df_diabetes.iloc[:, 0:-1]
y = df_diabetes.iloc[:, -1]


bayes_search_diabetes_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=xgboost_params,
                                n_iter=int(n_iter_BS),
                                #random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_diabetes_model.fit(X, y)

best_BS_f1_diabetes = bayes_search_diabetes_model.best_score_
diabetes_best_estimator = bayes_search_diabetes_model.best_estimator_

print(f"Best f1 score: {best_BS_f1_diabetes}")
diabetes_best_estimator

Best f1 score: 0.6628991819261374


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000282CD93E310>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncod...
                               importance_type=None,
                               interaction_constraints=None,
                               lambda=0.0009765625,
                               learning_rate=0.08036681855037739, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=3,
                               max_leaves=None, min_child_weight=None,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=None,
                               n_jobs=None, num_parallel_tree=None, ...))])

## Penguins dataset

In [36]:
X = df_penguins.iloc[:, 0:-1]
y = df_penguins.iloc[:, -1]


bayes_search_penguins_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=xgboost_params,
                                n_iter=int(n_iter_BS),
                                #random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_penguins_model.fit(X, y)

best_BS_f1_penguins = bayes_search_penguins_model.best_score_
penguins_best_estimator = bayes_search_penguins_model.best_estimator_

print(f"Best f1 score: {best_BS_f1_penguins}")

penguins_best_estimator

Best f1 score: 1.0


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000282CD8F5A50>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncod...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               lambda=0.0009765625, learning_rate=0.3,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=9, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, ...))])

## Tunability

### (3.3) Measuring Overall Tunability of a ML Algorithm

In [37]:
d_BS_credit = -sigma_star_row['credit_f1'].iloc[0] + best_BS_f1_credit
d_BS_college = -sigma_star_row['college_f1'].iloc[0] + best_BS_f1_college
d_BS_diabetes = -sigma_star_row['diabetes_f1'].iloc[0] + best_BS_f1_diabetes
d_BS_penguins = -sigma_star_row['penguins_f1'].iloc[0] + best_BS_f1_penguins

d_BS = pd.Series([d_BS_credit, d_BS_college, d_BS_diabetes, d_BS_penguins])
print(d_BS)
print(f"Mean: {d_BS.mean()}")

0    0.013388
1    0.007429
2    0.011536
3    0.010056
dtype: float64
Mean: 0.010602297917069514


### Models saving

In [38]:
from joblib import dump

dump(bayes_search_college_model, '../../data/BSmodels/XGBoost/bayes_search_college_model.joblib')
dump(bayes_search_credit_model, '../../data/BSmodels/XGBoost/bayes_search_credit_model.joblib')
dump(bayes_search_diabetes_model, '../../data/BSmodels/XGBoost/bayes_search_diabetes_model.joblib')
dump(bayes_search_penguins_model, '../../data/BSmodels/XGBoost/bayes_search_penguins_model.joblib')

['../../data/BSmodels/XGBoost/bayes_search_penguins_model.joblib']